In [ ]:
# Importy bibliotek
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns; sns.set()

import matplotlib.pylab as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Pliki CSV umieszczone w folderze data
uczestnicy = pd.read_csv('data/PPK_Uczestnicy.csv', sep=';')
pracodawcy = pd.read_csv('data/PPK_Pracodawcy.csv',sep=';')

# Mergowanie dwóch csv
df = pd.merge(
    uczestnicy,
    pracodawcy,
    how='left',
    left_on='EMPL_ID',
    right_on='ID'
)
df.info()
cp = df

In [ ]:
# cp to kopia przeczytana
df = cp


po_plci = df.SEX.value_counts(sort=True)
po_plci.plot.pie(subplots=True, autopct="%d%%", labels=["Mężczyźni","Kobiety","Inni/Inne"])
#Age to int

def to_int(age):
    age = age[:age.find(",")]
    return int(age)


df["AGE"] = df["AGE"].apply(to_int)
#dziele na pudelka z [30,40) itp
def to_box(age):
    return (age//10) * 10
df["AGE_BOX"] = df["AGE"].apply(to_box)
# Dropowanie niepotrzebnych kolumn
df.drop(columns=['MEMBER_ID','EMPL_ID', 'WORK_START', 'WORK_STOP', 'LOGICAL_FACTOR_1','LOGICAL_FACTOR_2','ID','PKD_CODE','PPK_BANK','NUMERICAL_VALUE'], inplace=True)
# zamian danych kategorycznych na dummy variables - tworzymy nową kolumnę na każdy możliwy output kolumny, porównaj poprzednią i następną komórkę
cp2 = df['SEX']

# df = pd.get_dummies(df, columns = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP'], prefix = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP'])
# df['SEX'] = cp2

# find duration of sign itp
def find_period(data):
    start, stop = data
    if type(stop) is float:
        stop = datetime.now().strftime("%Y-%m-%d")
    
    start_year, start_month, start_day = map(int, start.split('-'))
    stop_year, stop_month, stop_day = map(int, stop.split('-'))
    
    days = (stop_year - start_year)*365
    days += (stop_month - start_month)*30 if stop_month > start_month else (start_month - stop_month)*30
    days += stop_day - start_day if stop_day > start_day else start_day - stop_day
    
    return days


df['DURATION'] = list(zip(df['CREATED_AT'], df['RESIGN_DATE']))

df['DURATION'] = df['DURATION'].apply(find_period) 

df['RESIGNED'] = ~df['RESIGN_DATE'].isna()

df.drop(columns = ['SIGN_DATE', 'RESIGN_DATE'], inplace=True)
# z tym nie wiem co mam zrobic, mozna wyjebac albo cos mądrego zrobic
df.drop(columns=["CREATED_AT", "UOZ_START_DATE", "UOP_SIGN_DATE", 'REGION_CODE'], inplace=True)
print(df.info())
print(df.describe())

In [ ]:
# Grouping other nationalities into one category '0'
unique_nat = df['NATIONALITY'].unique()

# df.loc[df.NATIONALITY != 187, 'NATIONALITY'] = 0
for val in unique_nat:
    suma = ( df['NATIONALITY'].values == val ).sum()
    if suma < 1000:
        df.loc[df.NATIONALITY == val, 'NATIONALITY'] = 0

In [ ]:
df['NATIONALITY'].value_counts(sort=True)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(18,10))

sns.barplot(x = df['SEX'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[0][0])
ax[0][0].set_facecolor('#f5f5f5')
ax[0][0].set_ylim(0,1)
ax[0][0].set_xlabel(None)
ax[0][0].set_title('Gender')

sns.barplot(x = df['AGE_BOX'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[0][1])
ax[0][1].set_facecolor('#f5f5f5')
ax[0][1].tick_params(labelleft=False)
ax[0][1].set_ylim(0,1)
ax[0][1].set_ylabel(None)
ax[0][1].set_xlabel(None)
ax[0][1].set_title('WIEK')

sns.barplot(x = df['NATIONALITY'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[0][2])
ax[0][2].set_facecolor('#f5f5f5')
ax[0][2].tick_params(labelleft=False)
ax[0][2].set_ylim(0,1)
ax[0][2].set_ylabel(None)
ax[0][2].set_xlabel(None)
ax[0][2].set_title('Po narodowosci')

# HAS\_AE - Czy pracownik otrzymuje wpłatę dodatkową od pracodawcy?
sns.barplot(x = df['HAS_AE'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[1][0])
ax[1][0].set_facecolor('#f5f5f5')
ax[1][0].set_ylim(0,1)
ax[1][0].set_xlabel(None)
ax[1][0].set_title('wplaty od pracodawcy')

# HAS\_AW - Czy pracownik wpłaca własną dodatkową wpłatę?
sns.barplot(x = df['HAS_AW'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[1][1])
ax[1][1].set_facecolor('#f5f5f5')
ax[1][1].tick_params(labelleft=False)
ax[1][1].set_ylim(0,1)
ax[1][1].set_xlabel(None)
ax[1][1].set_title('własna dodatkowa wpłata')

# HAS\_IP - Czy pracownik otrzymał wpłatę powitalną?
sns.barplot(x = df['HAS_IP'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[1][2])
ax[1][2].set_facecolor('#f5f5f5')
ax[1][2].tick_params(labelleft=False)
ax[1][2].set_ylim(0,1)
ax[1][2].set_xlabel(None)
ax[1][2].set_title('Wpłata powitalna')

# PPK\_STAGE - w ktorym stadium pkk dolaczyl
sns.barplot(x = df['PPK_STAGE'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[2][0])
ax[2][0].set_facecolor('#f5f5f5')
ax[2][0].tick_params(labelleft=False)
ax[2][0].set_ylim(0,1)
ax[2][0].set_xlabel(None)
ax[2][0].set_title('Stadium ppk')

#COMPANY\_SIZE\_A - do 10 pracownikow
sns.barplot(x = df['COMPANY_SIZE'], y = df['IS_SUSPENDED'], color='#488ab5', ci=None, ax=ax[2][1])
ax[2][1].set_facecolor('#f5f5f5')
ax[2][1].tick_params(labelleft=False)
ax[2][1].set_ylim(0,1)
ax[2][1].set_xlabel(None)
ax[2][1].set_title('Wielkosc firmy (ilosc osob)')

plt.tight_layout(pad=4.0)


In [ ]:
print(df.loc[df.NATIONALITY, 'NATIONALITY'].count())
ilosc = df.loc[df.NATIONALITY, 'NATIONALITY'].count()
# Grouping other nationalities into one category '0'


# df.loc[df.NATIONALITY != 187, 'NATIONALITY'] = 0
for val in unique_nat:
    suma = ( df['NATIONALITY'].values == val ).sum()
    if suma < 1000:
        df.loc[df.NATIONALITY == val, 'NATIONALITY'] = 0